Подключение необходимых библиотек

In [1]:
import pm4py
import json
import os
from pm4py.objects.log.importer.xes import importer as xes_importer
import numpy as np
from math import sqrt

from sklearn.metrics import mean_squared_error, precision_score, recall_score, mean_absolute_error
from scipy.spatial import distance

from import_data import read_data_equisize, read_data_equitemp, determine_cutoff_point, ActivityPair
from forecasting import ARf, ARIMAf, HWf, NAVf, GARCHf
from operations import calculate_entropic_relevance

import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'pm4py'

Инициализация необходимых параметров

In [ ]:
############
# Parameters

dataset = 'italian'
agg_type = 'equisize' # возможен параметр 'equitemp'
no_pairs = 0
horizon = 25
no_intervals = 75
no_folds = 10
no_intervals_all = 100

Считывание лога

In [ ]:
variant = xes_importer.Variants.ITERPARSE
paras = {variant.value.Parameters.MAX_TRACES: 1000000000}
log = xes_importer.apply(dataset + '.xes', parameters=paras)

Кодирование активностей в целые числа

In [ ]:
activity_names = pm4py.get_attribute_values(log, 'concept:name')
no_act = len(activity_names)
act_map = {}
reverse_map = {}
for a, value in enumerate(activity_names.keys()):
    act_map[value] = a
    reverse_map[a] = value

Добавление фиктивных активностей "start" и "end"

In [ ]:
act_map['start'] = no_act
act_map['end'] = no_act + 1
reverse_map[no_act] = 'start'
reverse_map[no_act+1] = 'end'
no_act += 2
print('Activity encoding:', act_map)

Создание пар активностей существующих переходов внутри путей лога

In [ ]:
apairs = []
for t, trace in enumerate(log):
    for e, event in enumerate(trace):
        if e == len(trace) - 1:
            continue
        ap = ActivityPair(event['concept:name'], trace[e+1]['concept:name'], trace[e+1]['time:timestamp'], event, trace[e+1], t)
        apairs.append(ap)

In [ ]:
class ActivityPair:
    def __init__(self, a1, a2, timestamp, event, event2, trace_no):
        self.a1 = a1
        self.a2 = a2
        self.timestamp = timestamp
        self.event = event
        self.event2 = event2
        self.trace_no = trace_no

    def __str__(self):
        return self.a1 + ' before ' + self.a2 + ' at ' + str(self.timestamp)

    def __gt__(self, other):
        if self.timestamp > other.timestamp:
            return True
        else:
            return False

Сортировка переходов по времени и вывод их количества на экран

In [ ]:
sorted_aps = sorted(apairs)
print('#DFs:', len(sorted_aps))

Разбиение лога на интервалы и нахождение для каждого интервала матрицы числа переходов

In [ ]:
print('Creating DFG matrix')
if agg_type == 'equisize':
    dfg_time_matrix_org, interval_timings = read_data_equisize(no_intervals_all, sorted_aps, act_map, log, dataset)
else:
    dfg_time_matrix_org, interval_timings = read_data_equitemp(no_intervals_all, sorted_aps, act_map, log, dataset)

In [ ]:
def read_data_equisize(no_intervals, sorted_aps, act_map, log, dataset):
    interval_width = int(len(sorted_aps) / no_intervals)
    no_act = len(act_map.keys())

    dfg_time_matrix = np.zeros([no_intervals, no_act, no_act], dtype=int)

    interval_timing = []

    no_events_sums = 0   # число событий всего в интервале
    no_events_logs = 0   # число событий в записанном интервале лога
    no_dfs = 0   #

    shadow_dict = {}
    shadow_event_l = 0
    logs_finished = 0
    log_progression = {}
    for i in range(0, no_intervals): #проходим по каждому индексу интервала
        print('Interval ', i+1, '/', no_intervals)
        lower_bound = i * interval_width  # номер самой первого перехода из интервала
        upper_bound = (i+1) * interval_width # номер самого последнего перехода из интервала
        if i == (no_intervals - 1):
            upper_bound = len(sorted_aps) # если у нас последний интервал, то последний переход это последняя пара из лога

        dfs = sorted_aps[lower_bound:upper_bound] # берем срез переходов ActivePair из текущего интервала
        no_dfs += len(dfs) #текущее число обраотанных переходов

        print('#DFS:', len(dfs))

        empty_mat = np.zeros([no_act, no_act], dtype=float) #создаем пустую матрицу по числу активностей

        # For output
        filtered_events = {}
        start = Event()
        end = Event()
        start['concept:name'] = str(act_map['start'])
        end['concept:name'] = str(act_map['end'])
        highest = datetime(1970, 1, 1, tzinfo=pytz.UTC)
        lowest = datetime(2050, 1, 1, tzinfo=pytz.UTC)

        log_dfs = {}
        for df in dfs:  # идем по срезу пар из ActivePair
            if df.trace_no not in log_dfs.keys():
                log_dfs[df.trace_no] = [] # добавляем все попадающиеся новые трейсы в словарь
            log_dfs[df.trace_no].append(df) #добавляем в ключ трейса все его пары переходов ActivePair

        for trace_no, dfss in log_dfs.items(): # по порядку вытаскиевам трейс и все его попавшие переходы
            # print('\nTrace:', trace_no)
            sorted_dfs = sorted(dfss) # сортируем по времени все переходы в каждом трейсе
            filtered_events[trace_no] = []
            for df in sorted_dfs: # каждая пара в отсортированном массиве пар текущего трейса
                # print(df)
                filtered_events[trace_no].append(df.event) # упорядоченные пары добавляются в список
                no_events_sums += 1 # число событий в трейсе
            filtered_events[trace_no].append(sorted_dfs[len(sorted_dfs) - 1].event2) # так как перед этим добалялись 1ые события из пар,
                # то последнее событие оказалось пропущенным, добавялем его (не синтетическое end)

            no_events_sums += 1

        for trace_no, events in filtered_events.items(): # добавляем синтетические start и end  в каждый трейс
            empty_mat[act_map['start'], act_map[events[0]['concept:name']]] += 1
            empty_mat[act_map[events[-1]['concept:name']], act_map['end']] += 1

        # Export filtered events to interval event logs
        new_log = EventLog()
        no_eve = 0
        for t, trace in enumerate(log): # идем по первоначальному логу по трейсам
            if i < 50: # странная ситуевина, видимо что-то вспомогательное, не обращаем внимание
                if t not in shadow_dict.keys():
                    shadow_dict[t] = []
            new_trace = Trace()
            for trace_no, events in filtered_events.items(): # извлекаем из списка с путями номер трейса(индекс списка) и список событий
                if t == trace_no: # если номер трейса изначального лога совпал с номером трейса отфильтрованного лога
                    for event in trace: # то для каждого события в первоначальном трейсе
                        if event in events: # если событие из первоначального трейса попало в разделение по интервалам
                            if event['time:timestamp'] < lowest: # отслеживаем минимум времени интервала и максимум
                                lowest = event['time:timestamp']
                            if event['time:timestamp'] > highest:
                                highest = event['time:timestamp']
                            new_event = Event() # создаем событие
                            new_event['concept:name'] = str(act_map[event['concept:name']]) # преобразуем название в число
                            new_trace.append(new_event) # добавляем событие в трейс
                            no_events_logs += 1 # инкрементируем число событий в интервале лога
                            no_eve += 1
                            if i < 50:
                                shadow_dict[t].append(new_event)
                                shadow_event_l += 1
            if len(new_trace) > 0:
                # new_trace.append(end)
                new_log.append(new_trace)

        exporter.apply(new_log, './logs/' + dataset + '_log_interval_' + str(i) + '-'
                       + str(no_intervals) + '_equisize.xes')

        print('#Events:', no_eve)
        print('#Events log:', no_events_logs)
        print('#Events shadow low:', shadow_event_l)
        for act_pair in dfs: # расчитываем матрицу переходов для текущего интервала
            a1 = act_map[act_pair.a1]
            a2 = act_map[act_pair.a2]
            empty_mat[a1, a2] += 1

        dfg_time_matrix[i] = empty_mat # добавляем в индекс интервала соответствующую ему матрицу переходов

        interval_timing.append((lowest, highest))

    print('Event sums:', no_events_sums)
    print('Event logs:', no_events_logs)
    print('#DFS:', no_dfs)
    print('Logs finished:', logs_finished)
    return dfg_time_matrix, interval_timing

In [ ]:
def read_data_equitemp(no_intervals, sorted_aps, act_map, log, dataset):
    timestamps = pm4py.get_attribute_values(log, 'time:timestamp')
    print('Earliest:', min(timestamps))
    print('Latest:', max(timestamps))
    interval_length = (max(timestamps) - min(timestamps)) / no_intervals
    print('Interval length:', interval_length)

    no_act = len(act_map.keys())

    dfg_time_matrix = np.zeros([no_intervals, no_act, no_act], dtype=int)

    interval_timing = []
    no_events_sums = 0
    no_events_logs = 0
    no_dfs = 0
    for i in range(0, no_intervals):
        print('Interval ', i, '/', no_intervals)
        lower_bound = min(timestamps) + i * interval_length
        if i == (no_intervals - 1):
            upper_bound = min(timestamps) + (i + 1) * interval_length * 2
        else:
            upper_bound = min(timestamps) + (i + 1) * interval_length
        lb = lower_bound
        ub = upper_bound
        print(lb)
        print(ub)

        dfs = []
        empty_mat = np.zeros([no_act, no_act], dtype=float)

        filtered_events = {}
        start = Event()
        end = Event()
        start['concept:name'] = str(act_map['start'])
        end['concept:name'] = str(act_map['end'])
        highest = datetime(1970, 1, 1, tzinfo=pytz.UTC)
        lowest = datetime(2050, 1, 1, tzinfo=pytz.UTC)

        count = 0
        for df in sorted_aps:
            if ub > df.event2['time:timestamp'] >= lb:# and ub > df.event['time:timestamp'] >= lb:
                dfs.append(df)

        no_dfs += len(dfs)

        log_dfs = {}
        for df in dfs:
            if df.trace_no not in log_dfs.keys():
                log_dfs[df.trace_no] = []
            log_dfs[df.trace_no].append(df)

        for trace_no, dfss in log_dfs.items():
            # print('\nTrace:', trace_no)
            sorted_dfs = sorted(dfss)
            filtered_events[trace_no] = []
            for df in sorted_dfs:
                # print(df)
                filtered_events[trace_no].append(df.event)
                no_events_sums += 1
            filtered_events[trace_no].append(sorted_dfs[len(sorted_dfs)-1].event2)
            no_events_sums += 1

        print('#traces:', len(log_dfs))

        for trace_no, events in filtered_events.items():
            empty_mat[act_map['start'], act_map[events[0]['concept:name']]] += 1
            empty_mat[act_map[events[-1]['concept:name']], act_map['end']] += 1

        # Export filtered events to interval event logs
        new_log = EventLog()
        no_eve = 0
        for t, trace in enumerate(log):
            new_trace = Trace()
            # new_trace.append(start)
            for trace_no, events in filtered_events.items():
                if t == trace_no:
                    for event in trace:
                        if event in events:
                            if event['time:timestamp'] < lowest:
                                lowest = event['time:timestamp']
                            if event['time:timestamp'] > highest:
                                highest = event['time:timestamp']
                            new_event = Event()
                            new_event['concept:name'] = str(act_map[event['concept:name']])
                            new_trace.append(new_event)
                            no_events_sums += 1
                            no_eve += 1
            if len(new_trace) > 0:
                # new_trace.append(end)
                new_log.append(new_trace)
        exporter.apply(new_log, './logs/' + dataset + '_log_interval_' + str(i) + '-'
                       + str(no_intervals) + '_equitemp.xes')

        # print('no eve:', no_eve)
        for act_pair in dfs:
            a1 = act_map[act_pair.a1]
            a2 = act_map[act_pair.a2]
            empty_mat[a1, a2] += 1

        dfg_time_matrix[i] = empty_mat
        interval_timing.append((lowest, highest))
    print('Event sums:', no_events_sums)
    print('Event logs:', no_events_logs)
    print('#DFS:', no_dfs)

    return dfg_time_matrix, interval_timing


def determine_cutoff_point(act_map, dfg_time_matrix, no_pairs):
    counts = []
    for act, a in act_map.items():
        for act2, a2 in act_map.items():
            counts.append(np.sum(dfg_time_matrix[:, a, a2]))
    fc = Counter(counts)
    no_found = 0
    cutoff = 0
    for sum in sorted(fc.keys(), reverse=True):
        cutoff = sum
        no_found += fc[sum]
        if no_found > no_pairs:
            break
    print('CUTOFF:', cutoff, '#pairs:', no_found)

    return cutoff

In [ ]:
with open(f'dfg_time_matrix_{dataset}_{agg_type}.npy', 'wb') as f:
    np.save(f, dfg_time_matrix_org)

Нахождение величины числа переходов, чтобы оставить наиболее частые относительно числа отображаемых переходов заданных в параметры no_pairs

In [ ]:
if no_pairs == 0:
    cutoff = 0
else:
    cutoff = determine_cutoff_point(act_map, dfg_time_matrix_org, no_pairs)

Обрезка матрицы переходов берем no_intervals число интервалов из no_intervals_all интервалов

In [ ]:
# reduce matrix according to parameter settings
dfg_time_matrix = dfg_time_matrix_org[:no_intervals, ::, ::]

Инициализация переменных для хранения результатов предсказаний

In [ ]:
techniques = ['nav', 'ar1', 'ar2', 'ar4', 'arima211', 'arima212', 'hw', 'garch']

dfg_result_matrix = {}
dfg_actual_matrix = np.zeros([no_act, no_act, no_folds, horizon], dtype=int)

for technique in techniques:
    dfg_result_matrix[technique] = np.zeros([no_act, no_act, no_folds, horizon], dtype=float)


Классы обертки для преддиктивных моделей

In [ ]:
class NAVf:

    class Fitted:

        def __init__(self, x):
            self.x = x

        def forecast(self, horizon):
            mean = np.mean(self.x)
            # print('Naive:', np.full(shape=(1, horizon), fill_value=mean))
            return np.full(shape=(1, horizon), fill_value=mean)

    def __init__(self):
        pass

    def fit(self, x, horizon):
        return self.Fitted(x).forecast(horizon)[0]


class ARf:

    def __init__(self, d):
        self.d = d

    def fit(self, x, horizon):
        fit = AR(x, lags=self.d, old_names=False).fit()
        point_forecast = fit.get_prediction(len(x), len(x)+horizon-1)

        # print(f'AR {self.d}:', point_forecast.predicted_mean)
        # print(point_forecast.conf_int(0.05))

        return point_forecast.predicted_mean


class ARIMAf:

    def __init__(self, p, d, q):
        self.p = p
        self.d = d
        self.q = q

    def fit(self, x, horizon):
        fit = ARIMA(x, order=(self.p, self.d, self.q)).fit()
        point_forecast = fit.get_forecast(horizon)
        conf_int = fit.conf_int()

        #print('ARIMA:', point_forecast.predicted_mean)
        #print(point_forecast.conf_int())
        return point_forecast.predicted_mean

class HWf:

    def __init__(self):
        pass

    def fit(self, x, horizon):
        fit = HW(x, initialization_method='estimated').fit()

        # print('HW', fit.forecast(horizon))
        return fit.forecast(horizon)


class GARCHf:

    def __init__(self):
        pass

    def fit(self, x, horizon):
        garch = ARCH(x)
        model = garch.fit(disp='off')
        y_hat = model.forecast(horizon=horizon)

        pred = y_hat.mean.iloc[[-1]].values[0]
        var = np.sqrt(y_hat.variance.iloc[[-1]].values) * 1.96
        conf_low = pred + var[0]
        conf_high = pred - var[0]
        return pred


In [ ]:
Обучение на интервалах, получение предсказаний

In [ ]:
chosen_pairs = set()
# forecast DFs of all activity pairs
for act, a in act_map.items():
    for act_2, a2 in act_map.items():

        # by default only time series with at least 1 DF will be selected
        if np.sum(dfg_time_matrix[:, a, a2]) > cutoff:
            print('DFG', act, 'and', act_2)
            chosen_pairs.add((a, a2))

            # get DF
            array = dfg_time_matrix[:, a, a2] # срез числа появлений пары событий в каждом интервале

            techniques = dict()
            techniques['nav'] = NAVf()
            # techniques['ar1'] = ARf(1)
            techniques['ar2'] = ARf(2)
            techniques['ar4'] = ARf(4)
            # techniques['arima211'] = ARIMAf(2, 1, 1)
            techniques['arima212'] = ARIMAf(2, 1, 2)
            techniques['hw'] = HWf()
            techniques['garch'] = GARCHf()

            # cross-validation is applied
            for fold in range(0, no_folds):
                # offset for cross-validation
                offset = - fold - horizon
                x = array[:offset] # обрезаем срез числа появлений на обучающую и тестовую выборку

                if fold == 0:
                    y = array[offset:]
                else:
                    y = array[offset:(offset + horizon)]

                # store actual
                dfg_actual_matrix[a, a2, fold] = y

                for technique, implement in techniques.items():
                    y_pred = []
                    my_x = np.copy(x)
                    ######
                    print('Массив для обучения для вершин ', a, a2)
                    print(type(my_x))
                    print(my_x)
                    ######
                    try:
                        # predict horizon steps ahead
                        y_hat = implement.fit(my_x, horizon)
                        y_pred = y_hat
                        dfg_result_matrix[technique][a, a2, fold] = y_pred
                    except:
                        dfg_result_matrix[technique][a, a2, fold] = np.full((horizon, ), 100000000)


Подключение работы библиотеки для расчета энтропии результатов

In [ ]:
def calculate_entropic_relevance(log_name, dfg_name):
    try:
        args = []
        args.append('java')  # Запуск Java
        args.append('-cp')  # Указание classpath
        # Указываем все необходимые JAR-файлы через точку с запятой (для Windows)
        args.append('jbpt-pm-entropia-1.6.jar;commons-cli-1.9.0.jar')
        args.append('org.jbpt.pm.tools.QualityMeasuresCLI')  # Указываем основной класс
        args.append('-r')  # Параметры для основного класса
        args.append('-s')
        args.append('-rel')
        logstring = log_name + '.xes'  # Имя лог-файла
        args.append(logstring)
        predstring = dfg_name + '.json'  # Имя JSON-файла
        args.append('-ret')
        args.append(predstring)

        # Запуск команды и получение результата
        result = subprocess.check_output(args)
        result_s = float(result)  # Преобразуем результат в число
    except Exception as e:
        result_s = 'NA'
        print("Error:", e)  # Вывод ошибки, если она возникла

    return result_s

Оценка полученных результатов

In [ ]:
with open(f'results_{dataset}_nopairs_{no_pairs}_nointervals_{str(no_intervals)}_{agg_type}.csv', 'w') as technique_fold_results:
    technique_fold_results.write('intervals,technique,fold,horizon,cosine,rmse,er_pred,er_actual\n')

    for technique in techniques:
        print(f'Technique {technique}')
        dfg_result_matrix_ar = dfg_result_matrix[technique]
        results_selected = np.zeros((len(chosen_pairs), no_folds, horizon))
        actual_selected = np.zeros((len(chosen_pairs), no_folds, horizon))

        for p, pair in enumerate(chosen_pairs):
            results_selected[p] = dfg_result_matrix_ar[pair[0], pair[1], ::, ::]
            actual_selected[p] = dfg_actual_matrix[pair[0], pair[1], ::, ::]

        for h in range(0, horizon):
            for fold in range(0, no_folds):
                # print(f'Fold {fold} - horizon {h}')

                #####################
                # Forecast DFG output
                nodes = []
                for act_name, act_code in act_map.items():
                    out_freq = int(np.sum(dfg_result_matrix_ar[act_code, ::, fold, h]))
                    in_freq = int(np.sum(dfg_result_matrix_ar[::, act_code, fold, h]))

                    if act_name == 'start':
                        nodes.append({'label': str(act_code), 'freq': out_freq, 'id': act_code})
                    else:
                        if in_freq > 0:
                            nodes.append({'label': str(act_code), 'freq': in_freq, 'id': act_code})

                arcs = []
                for a in range(0, len(act_map)):
                    for a2 in range(0, len(act_map)):
                        if int(dfg_result_matrix_ar[a, a2, fold, h]) > 0:
                            arcs.append({'from': a, 'to': a2, 'freq': int(dfg_result_matrix_ar[a, a2, fold, h])})

                # calculate entropic relevance
                dfg_file = {'nodes': nodes, 'arcs': arcs}
                r = json.dumps(dfg_file, indent=1)
                with open(f'temp.json', 'w') as dfg_write_file:
                    dfg_write_file.write(r)

                offset = no_intervals - horizon + h - fold
                er_technique = calculate_entropic_relevance(f'./logs/{dataset}_log_interval_{offset}-{no_intervals_all}_{agg_type}', 'temp')


                ###################
                # Actual DFG output
                nodes = []
                for act_name, act_code in act_map.items():
                    out_freq = int(np.sum(dfg_actual_matrix[act_code, ::, fold, h]))
                    in_freq = int(np.sum(dfg_actual_matrix[::, act_code, fold, h]))

                    if act_name == 'start':
                        nodes.append({'label': str(act_code), 'freq': out_freq, 'id': act_code})
                    else:
                        if in_freq > 0:
                            nodes.append({'label': str(act_code), 'freq': in_freq, 'id': act_code})

                arcs = []
                for a in range(0, len(act_map)):
                    for a2 in range(0, len(act_map)):
                        if int(dfg_actual_matrix[a, a2, fold, h]) > 0:
                            arcs.append({'from': a, 'to': a2, 'freq': int(dfg_actual_matrix[a, a2, fold, h])})

                # calculate entropic relevance
                dfg_file = {'nodes': nodes, 'arcs': arcs}
                r = json.dumps(dfg_file, indent=1)
                with open(f'temp.json', 'w') as dfg_write_file:
                    dfg_write_file.write(r)

                offset = no_intervals - horizon + h - fold
                er_actual = calculate_entropic_relevance(f'./logs/{dataset}_log_interval_{offset}-{no_intervals_all}_{agg_type}', 'temp')

                # store results
                results = np.reshape(results_selected[::, fold, h], (1, len(chosen_pairs)))
                actuals = np.reshape(actual_selected[::, fold, h], (1, len(chosen_pairs)))

                cosine = distance.cosine(results, actuals)
                rmse = sqrt(mean_squared_error(actuals, results))

                technique_fold_results.write(f'{no_intervals},{technique},{str(fold)},{str(h)},{cosine},{rmse},{er_technique},{er_actual}\n')
